In [96]:
import pandas as pd
import astropy.coordinates as apc
import os
from datetime import datetime
import numpy as np
from astroquery.jplhorizons import Horizons
from astropy.io import fits
from astropy import units as u
from astropy.coordinates import Angle

In [107]:
dfs = pd.read_html("http://www.johnstonsarchive.net/astro/tnodiam.html")

In [108]:
df_TNO=dfs[1]

In [109]:
df_TNO = df_TNO[['number/name', 'prov. des.','type','effectivediameter (km)','primarydiameter (km)','albedo','meth','source']]

In [110]:
tno_types = ["CUB","HAU","SDO","TNO"]

In [111]:
df_TNO = df_TNO.dropna(subset=['effectivediameter (km)', 'albedo'])

In [102]:
df_TNO_johnston = df_TNO.loc[df_TNO['type'].isin(tno_types)]

In [78]:
TNO_johnston_names = np.array(df_TNO_johnston['prov. des.'])

In [82]:
TNO_dict = {}
for i in range(len(TNO_johnston_names)):
    obj = Horizons(id=TNO_johnston_names[i], location='568',epochs={'start':'2022-01-31', 'stop':'2022-02-15','step':'7d'})
    eph = obj.ephemerides()
    datetime_str_array=np.array(eph['datetime_str'])
    ra_array=np.array(eph['RA'])
    dec_array=np.array(eph['DEC'])
    TNO_dict[TNO_johnston_names[i]]={
        "datetime_str": datetime_str_array,
        "RA" : ra_array,
        "DEC" : dec_array
    }

In [113]:
RA_0131_johnston =[]
DEC_0131_johnston = []
for i in TNO_johnston_names:
    RA_0131_johnston = np.append(RA_0131_johnston,TNO_dict[i]["RA"][0])
    DEC_0131_johnston = np.append(DEC_0131_johnston,TNO_dict[i]["DEC"][0])
    
df_TNO_johnston['RA_0131'] = RA_0131_johnston
df_TNO_johnston['DEC_0131'] = DEC_0131_johnston

In [114]:
df_TNO_johnston_dec013122_cuts = df_TNO_johnston.loc[df_TNO_johnston['DEC_0131']>-15.0]

In [38]:
TNO_DES_cat = fits.open("../Desktop/planet9/destno_main.fits")

In [39]:
TNO_DES_names = TNO_DES_cat[1].data["MPC"]
TNO_DES_mr = TNO_DES_cat[1].data["m_r"]
TNO_DES_Hr = TNO_DES_cat[1].data["H_r"]
TNO_DES_semi_a = TNO_DES_cat[1].data["a"]
TNO_DES_ecc = TNO_DES_cat[1].data["e"]
TNO_DES_dist = TNO_DES_cat[1].data["d"]

In [52]:
TNO_DES_cat_summary = np.column_stack([TNO_DES_names,TNO_DES_mr,TNO_DES_Hr,TNO_DES_semi_a,TNO_DES_ecc,TNO_DES_dist])

In [47]:
TNO_dict = {}
for i in range(len(TNO_DES_names)):
    obj = Horizons(id=TNO_DES_names[i], location='568',epochs={'start':'2022-01-31', 'stop':'2022-02-20','step':'7d'})
    eph = obj.ephemerides()
    datetime_str_array=np.array(eph['datetime_str'])
    ra_array=np.array(eph['RA'])
    dec_array=np.array(eph['DEC'])
    TNO_dict[TNO_DES_names[i]]={
        "datetime_str": datetime_str_array,
        "RA" : ra_array,
        "DEC" : dec_array
    }

In [66]:
RA_0131 =[]
DEC_0131 = []
for i in TNO_DES_names:
    RA_0131 = np.append(RA_0131,TNO_dict[i]["RA"][0])
    DEC_0131 = np.append(DEC_0131,TNO_dict[i]["DEC"][0])
    
TNO_DES_cat_0131DEC_cuts=TNO_DES_cat_summary[DEC_0131>-15.0]
df_TNO_DES_cat_0131DEC_cuts = pd.DataFrame(TNO_DES_cat_0131DEC_cuts,columns=['MPC','m_r','H_r','a','e','d'])

In [76]:
TNO_dict = {}
for i in range(len(TNO_DES_names)):
    obj = Horizons(id=TNO_DES_names[i], location='568',epochs={'start':'2022-04-30', 'stop':'2022-05-20','step':'7d'})
    eph = obj.ephemerides()
    datetime_str_array=np.array(eph['datetime_str'])
    ra_array=np.array(eph['RA'])
    dec_array=np.array(eph['DEC'])
    TNO_dict[TNO_DES_names[i]]={
        "datetime_str": datetime_str_array,
        "RA" : ra_array,
        "DEC" : dec_array
    }
    
RA_0430 =[]
DEC_0430 = []
for i in TNO_names:
    RA_0430 = np.append(RA_0430,TNO_dict[i]["RA"][0])
    DEC_0430 = np.append(DEC_0430,TNO_dict[i]["DEC"][0])
 
TNO_DES_cat_0430DEC_cuts = TNO_DES_cat_summary[DEC_0430>-15.0]
df_TNO_DES_cat_0430DEC_cuts = pd.DataFrame(TNO_DES_cat_0430DEC_cuts,columns=['MPC','m_r','H_r','a','e','d'])

ConnectionError: HTTPSConnectionPool(host='ssd.jpl.nasa.gov', port=443): Read timed out.

In [ ]:
RA_0430 =[]
DEC_0430 = []
for i in TNO_DES_names:
    RA_0430 = np.append(RA_0430,TNO_dict[i]["RA"][0])
    DEC_0430 = np.append(DEC_0430,TNO_dict[i]["DEC"][0])
 
TNO_DES_cat_0430DEC_cuts = TNO_DES_cat_summary[DEC_0430>-15.0]
df_TNO_DES_cat_0430DEC_cuts = pd.DataFrame(TNO_DES_cat_0430DEC_cuts,columns=['MPC','m_r','H_r','a','e','d'])

In [71]:
merged_cat_0131DEC_cuts = pd.merge(
    df_TNO_DES_cat_0131DEC_cuts,
    df_TNO_johnston,
    how="inner",
    on=None,
    left_on= 'MPC',
    right_on='prov. des.',
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

,MPC,m_r,H_r,a,e,d,number/name,prov. des.,type,effectivediameter (km),primarydiameter (km),albedo,meth,source
0,2001 QS322,23.235240549260432,6.9577385364553415,44.02284868132435,0.03873681149197331,42.462074641810744,NaN,2001 QS322,CUB,<200,NaN,>0.15 V,T,B09a
1,2001 QS322,23.235240549260432,6.9577385364553415,44.02284868132435,0.03873681149197331,42.462074641810744,NaN,2001 QS322,CUB,186±99/24,NaN,0.095±0.531/0.060 V,T,V14a
2,2014 UZ224,22.927009531335443,3.3035124421915896,108.43816821752827,0.6464038071630825,91.46793764494277,NaN,2014 UZ224,SDO,635±65/72,NaN,0.131±0.039/0.028,T,G17a
